# Maritime Choke Point Trends Monitor

The objective of this analysis is to examine the impact of the Red Sea Conflict on maritime trade statistics derived from AIS data.  

We process transit calls and estimated trade volume from the IMF's [PortWatch](https://portwatch.imf.org/) platform for key ports of interest, and then produce some charts to inspect trends and calculate percentage changes.

In [2]:
import os, sys
from os.path import join, expanduser

import requests
from tqdm.notebook import tqdm

import pandas as pd
import geopandas as gpd
from shapely.geometry import shape, Point
from datetime import datetime
import time

import git
git_repo = git.Repo(os.getcwd(), search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")
sys.path.append(join(git_root, 'src', 'red-sea-monitoring'))
from utils import *

# For plotting
from plotnine import *
from mizani.breaks import date_breaks
from mizani.formatters import date_format, percent_format, comma_format
import plotnine
import seaborn as sns
import matplotlib.pyplot as plt

plotnine.options.figure_size = (10, 8)

output_dir = r'C:\Users\WB514197\OneDrive - WBG\GOST_Deliverables\Red Sea\Data'
charts_dir = join(git_root, 'reports')

In [3]:
%load_ext autoreload
%autoreload 2

## Ports

In [4]:
ports = get_ports()

In [5]:
countries = ['Egypt', 'Yemen', 'Djibouti', 'Jordan', 'Saudi Arabia']

In [6]:
ports_sel = ports.loc[ports.country.isin(countries)].copy()

In [7]:
ports_sel.sort_values('country', inplace=True)

In [8]:
del(ports)

In [8]:
ports_sel.loc[:, "geometry"] = ports_sel.apply(lambda x: Point(x.lon, x.lat), axis=1)

In [9]:
ports_gdf = gpd.GeoDataFrame(ports_sel, geometry='geometry', crs='EPSG:4326')

In [10]:
# ports_gdf.explore()

In [11]:
# ports_gdf.to_file('ports_sel.geojson', driver='GeoJSON')
del(ports_gdf)

In [9]:
ports_red_sea = gpd.read_file(join(git_root, 'data', 'red_sea_ports.geojson'), driver='GeoJSON')
# ports_red_sea = pd.read_csv('red_sea_ports.csv')

In [10]:
# ports_red_sea.explore()

In [11]:
ports_red_sea[['country', 'portname', 'portid']]

,country,portname,portid
0,Djibouti,Djibouti,port294
1,Egypt,El-Adabiya,port321
2,Egypt,Safaga,port191
3,Egypt,North Ain Sukhna Port,port828
4,Egypt,As Suways,port71
5,Jordan,Al Aqabah,port19
6,Saudi Arabia,Duba Bulk Plant Tanker Terminal,port305
7,Saudi Arabia,Rabigh,port1081
8,Saudi Arabia,King Fahd Port,port570
9,Saudi Arabia,Duba,port304


### Trade Data

In [15]:
ports = list(ports_red_sea.portid)

In [17]:
df_ports = get_port_data(ports)

In [ ]:
df_ports.to_csv(join(output_dir, 'ports_data.csv'), index=False)

## Data Analysis

In [26]:
df = pd.read_csv(join(output_dir, 'ports_data.csv'))
df.date = pd.to_datetime(df.date)

In [27]:
df = df.loc[df.date>="2019-01-01"].copy()

In [23]:
df = df[['portname', 'portcalls_cargo', 'portcalls_tanker', 'portcalls', 'import_cargo', 'export_cargo', 'import_tanker', 'export_tanker', 'import', 'export', 'date']].copy()

In [29]:
# df = df.groupby('portname')[['portcalls_cargo', 'portcalls_tanker', 'portcalls', 'import_cargo', 'export_cargo', 'import_tanker', 'export_tanker', 'import', 'export', 'date']].rolling(7, center=True, min_periods=1, on='date').mean()
df = df.groupby('portname')[['portcalls_cargo', 'portcalls_tanker', 'portcalls', 'import_cargo', 'export_cargo', 'import_tanker', 'export_tanker', 'import', 'export', 'date']].resample('W-Mon', on='date').sum().reset_index()
# df.reset_index(inplace=True)
# df.drop('level_1', axis=1, inplace=True)
df.loc[:, "ymd"] = df.date.dt.strftime('%Y-%m-%d')
df.loc[:, "w"] = df.date.dt.strftime('%W')
# df.loc[:, "m"] = df.date.dt.strftime('%m')
# df.loc[:, "w"] = df.date.dt.strftime('%W')

In [30]:
df.head()

,portname,date,portcalls_cargo,portcalls_tanker,portcalls,import_cargo,export_cargo,import_tanker,export_tanker,import,export,ymd,w
0,Aden,2019-01-07,9,1,10,67364.559011,371.408892,0.000000,0.000000,67364.559011,371.408892,2019-01-07,01
1,Aden,2019-01-14,9,0,9,37879.939218,0.000000,0.000000,0.000000,37879.939218,0.000000,2019-01-14,02
2,Aden,2019-01-21,9,3,12,59428.392090,1476.816374,23018.373940,3160.095388,82446.766030,4636.911762,2019-01-21,03
3,Aden,2019-01-28,5,2,7,32601.162948,0.000000,2391.884286,0.000000,34993.047235,0.000000,2019-01-28,04
4,Aden,2019-02-04,11,2,13,53898.845185,411.297833,36085.859781,3160.095388,89984.704966,3571.393221,2019-02-04,05


### Plot Transit Calls Historical

In [31]:
# start_reference_date = "2022-01-01"
conflict_date = "2023-10-07"
crisis_date = "2023-11-17"

In [32]:
df_filt = df.loc[(df.date>="2023-01-01")].copy()

In [33]:
charts_by_port_dir = join(charts_dir, 'ports')
if not os.path.exists(charts_by_port_dir):
    os.makedirs(charts_by_port_dir, mode=0o777)

In [102]:
for port in df_filt.portname.unique():
    port_info = ports_red_sea.loc[ports_red_sea.portname==port].iloc[0]
    country = port_info.country
    port_id = port_info.portid
    country = ports_red_sea.loc[ports_red_sea.portname==port, "country"].values[0]
    df_port = df_filt.loc[df_filt.portname==port].copy()
    df_port = df_port.melt(id_vars='date', value_vars=['import', 'export'], var_name='direction', value_name='trade')
    df_port.loc[:, 'direction'] = df_port.direction.str.capitalize()
    p0 = (
    ggplot(df_port, aes(x="date", y="trade", fill="direction")) #
        + geom_bar(alpha=3/4, stat = "identity", position = "dodge2") #  fill="lightblue"
        # + geom_line(aes(x="date", y="export"), alpha=1, color="darkred")
        + geom_vline(xintercept=conflict_date, linetype="dashed", color = "black")
        + geom_vline(xintercept=crisis_date, linetype="dashed", color = "black")
        + labs(
            x="", y="Metric Tons", title=f"Weekly Trade Volume - {port}, {country}",
            fill="Trade Flow"
        )
        + theme_minimal() 
        + theme(text=element_text(family="Roboto"))
        + scale_x_datetime(breaks=date_breaks('1 month'), labels=date_format('%Y-%m'))
        + scale_y_continuous(labels=comma_format())
        # + scale_fill_brewer(type='qual', palette=1)
        + theme(axis_text_x=element_text(rotation=45, hjust=1))
        + theme(legend_position='bottom')
    )
    p0
    # p0.save(filename=join(charts_by_port_dir, f'estimated-trade-{port_id}.jpeg'), dpi=300)

In [34]:
# p0

Periods
- **Baseline**: 2021, 2022, 2023 (January 1st – October 6th)
- **Middle East Conflict**: 2023 (October 7th - November 16th)
- **Red Sea Crisis**: November 17th - January 31st, 2024

### Calculate Reference Values

In [133]:
start_reference_date = "2019-01-01"
conflict_date = "2023-10-07"
crisis_date = "2023-11-17"

In [134]:
df_ref = df.loc[(df.date>=start_reference_date)&(df.date<conflict_date)].copy()

In [135]:
df_ref = df_ref.groupby(['portname', 'w'])[['portcalls_cargo', 'portcalls_tanker', 'portcalls', 'import_cargo', 'export_cargo', 'import_tanker', 'export_tanker', 'import', 'export']].mean()

In [136]:
df_ref.reset_index(inplace=True)

In [137]:
df_ref.rename(columns={'portcalls_cargo': 'portcalls_cargo_ref', 'portcalls_tanker': 'portcalls_tanker_ref', 'portcalls': 'portcalls_ref', 'import_cargo': 'import_cargo_ref', 'export_cargo': 'export_cargo_ref', 'import_tanker': 'import_tanker_ref', 'export_tanker': 'export_tanker_ref', 'import': 'import_ref', 'export': 'export_ref'}, inplace=True)

In [138]:
df_filt = df.loc[(df.date>="2023-01-01")].copy()

In [139]:
df_filt = df_filt.merge(df_ref, on=["portname", "w"], how="left", validate="m:1")

In [148]:
# df_filt.loc[:, "export_pct_ch"] = df_filt.apply(lambda x: (x.export-x.export_ref) / (x.export_ref), axis=1)
# df_filt.loc[:, "import_pct_ch"] = df_filt.apply(lambda x: (x['import']-x.import_ref) / (x.import_ref), axis=1)

In [144]:
charts_by_port_dir = join(charts_dir, 'ports-ref')
if not os.path.exists(charts_by_port_dir):
    os.makedirs(charts_by_port_dir, mode=0o777)

In [146]:
for port in df_filt.portname.unique():
    port_info = ports_red_sea.loc[ports_red_sea.portname==port].iloc[0]
    country = port_info.country
    port_id = port_info.portid
    country = ports_red_sea.loc[ports_red_sea.portname==port, "country"].values[0]
    df_port = df_filt.loc[df_filt.portname==port].copy()
    df_port_copy = df_port.copy()
    df_port = df_port.melt(id_vars='date', value_vars=['import', 'export'], var_name='direction', value_name='trade')
    df_port.loc[:, 'direction'] = df_port.direction.str.capitalize()
    p0 = (
    ggplot(df_port_copy, aes(x="date", y="import_ref")) #
        + geom_smooth(mapping=aes(x="date", y="import_ref"), color='teal', size=0.4, alpha=3/4)
        + geom_smooth(mapping=aes(x="date", y="export_ref"), color='red', size=0.4, alpha=3/4)
        + geom_bar(mapping=aes(x="date", y="trade", fill="direction"), data=df_port, alpha=3/4, stat = "identity", position = "dodge2") #  fill="lightblue"
        # + geom_line(aes(x="date", y="export"), alpha=1, color="darkred")
        + geom_vline(xintercept=conflict_date, linetype="dashed", color = "black")
        + geom_vline(xintercept=crisis_date, linetype="dashed", color = "black")
        + labs(
            x="", y="Metric Tons", title=f"Weekly Trade Volume - {port}, {country}",
            fill="Trade Flow"
        )
        + theme_minimal() 
        + theme(text=element_text(family="Roboto"))
        + scale_x_datetime(breaks=date_breaks('1 month'), labels=date_format('%Y-%m'))
        + scale_y_continuous(labels=comma_format())
        # + scale_fill_brewer(type='qual', palette=1)
        + theme(axis_text_x=element_text(rotation=45, hjust=1))
        + theme(legend_position='bottom')
    )
    p0.save(filename=join(charts_by_port_dir, f'estimated-trade-{port_id}.jpeg'), dpi=300)

c:\WBG\Anaconda3\envs\rtmis\Lib\site-packages\plotnine\ggplot.py:587: PlotnineWarning: Saving 10 x 8 in image.
c:\WBG\Anaconda3\envs\rtmis\Lib\site-packages\plotnine\ggplot.py:588: PlotnineWarning: Filename: C:/Users/WB514197/Repos/red-sea-monitoring\reports\ports-ref\estimated-trade-port9.jpeg
c:\WBG\Anaconda3\envs\rtmis\Lib\site-packages\plotnine\stats\smoothers.py:330: PlotnineWarning: Confidence intervals are not yet implemented for lowess smoothings.
c:\WBG\Anaconda3\envs\rtmis\Lib\site-packages\plotnine\stats\smoothers.py:330: PlotnineWarning: Confidence intervals are not yet implemented for lowess smoothings.
c:\WBG\Anaconda3\envs\rtmis\Lib\site-packages\plotnine\ggplot.py:587: PlotnineWarning: Saving 10 x 8 in image.
c:\WBG\Anaconda3\envs\rtmis\Lib\site-packages\plotnine\ggplot.py:588: PlotnineWarning: Filename: C:/Users/WB514197/Repos/red-sea-monitoring\reports\ports-ref\estimated-trade-port18.jpeg
c:\WBG\Anaconda3\envs\rtmis\Lib\site-packages\plotnine\stats\smoothers.py:330: 